# 使⽤重复元素的⽹络（VGG）
AlexNet在LeNet的基础上增加了3个卷积层。但AlexNet作者对它们的卷积窗口、输出通道数和构
造顺序均做了⼤量的调整。虽然AlexNet指明了深度卷积神经⽹络可以取得出⾊的结果，但并没
有提供简单的规则以指导后来的研究者如何设计新的⽹络。我们将在本章的后续⼏节⾥介绍⼏种
不同的深度⽹络设计思路。
本节介绍VGG，它的名字来源于论⽂作者所在的实验室Visual Geometry Group [1]。 VGG提出了
可以通过重复使⽤简单的基础块来构建深度模型的思路。
## VGG块
VGG块的组成规律是：连续使⽤数个相同的填充为1、窗口形状为3 × 3的卷积层后接上⼀个步幅
为2、窗口形状为2 × 2的最⼤池化层。卷积层保持输⼊的⾼和宽不变，而池化层则对其减半。我
们使⽤vgg_block函数来实现这个基础的VGG块，它可以指定卷积层的数量num_convs和输出
通道数num_channels。

In [1]:
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn
def vgg_block(num_convs, num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels,kernel_size=3, padding=1, activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2,strides=2))
    return blk

## VGG网络
与AlexNet和LeNet⼀样， VGG⽹络由卷积层模块后接全连接层模块构成。卷积层模块串联数
个vgg_block，其超参数由变量conv_arch定义。该变量指定了每个VGG块⾥卷积层个数和
输出通道数。全连接模块则跟AlexNet中的⼀样。

现在我们构造⼀个VGG⽹络。它有5个卷积块，前2块使⽤单卷积层，而后3块使⽤双卷积层。第
⼀块的输出通道是64，之后每次对输出通道数翻倍，直到变为512。因为这个⽹络使⽤了8个卷积
层和3个全连接层，所以经常被称为VGG-11。 下⾯我们实现VGG-11。

In [2]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
def vgg(conv_arch):
    net = nn.Sequential()
    # 卷积层部分
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    # 全连接层部分
    net.add(nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
            nn.Dense(4096, activation='relu'), nn.Dropout(0.5),
            nn.Dense(10))
    return net
net = vgg(conv_arch)

下⾯构造⼀个⾼和宽均为224的单通道数据样本来观察每⼀层的输出形状。

In [3]:
net.initialize()
X = nd.random.uniform(shape=(1,1,224,224))
for layer in net:
    X = layer(X)
    print(layer.name, 'layer shape\t', X.shape)

sequential1 layer shape	 (1, 64, 112, 112)
sequential2 layer shape	 (1, 128, 56, 56)
sequential3 layer shape	 (1, 256, 28, 28)
sequential4 layer shape	 (1, 512, 14, 14)
sequential5 layer shape	 (1, 512, 7, 7)
dense0 layer shape	 (1, 4096)
dropout0 layer shape	 (1, 4096)
dense1 layer shape	 (1, 4096)
dropout1 layer shape	 (1, 4096)
dense2 layer shape	 (1, 10)


可以看到，每次我们将输⼊的⾼和宽减半，直到最终⾼和宽变成7后传⼊全连接层。与此同时，输
出通道数每次翻倍，直到变成512。因为每个卷积层的窗口⼤小⼀样，所以每层的模型参数尺⼨
和计算复杂度与输⼊⾼、输⼊宽、输⼊通道数和输出通道数的乘积成正⽐。 VGG这种⾼和宽减半
以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺⼨和计算复杂度。
## 获取数据和训练模型
因为VGG-11计算上⽐AlexNet更加复杂，出于测试的⽬的我们构造⼀个通道数更小，或者说更窄
的⽹络在Fashion-MNIST数据集上进⾏训练。

In [6]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)
# 除了使⽤了稍⼤些的学习率，模型训练过程与上⼀节的AlexNet中的类似
lr, num_epochs, batch_size, ctx = 0.05, 5, 128, d2l.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

## ⼩结
- VGG-11通过5个可以重复使⽤的卷积块来构造⽹络。根据每块⾥卷积层个数和输出通道数
的不同可以定义出不同的VGG模型。